# Getting the data

In [1]:
import numpy as np
import sys
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# import pandas
import json

# from sklearn.externals import joblib
import pickle
from importlib.resources import path
from pathlib import Path

import numpy as np
import pandas as pd
import sklearn.utils
from pandas import Int64Index, MultiIndex
from sklearn import metrics

In [2]:
import os
import importlib
import time
import pandas as pd

dirname = os.path.dirname('D:/fincrime-federated/')
os.chdir(dirname)
print(os.getcwd())

import submission_src.fincrime.solution_centralized as funcs
importlib.reload(funcs) 

start_time = time.time()

model_dir = 'D:/fincrime-federated/model/fincrime'
preds_format_path = 'D:/fincrime-federated/prediction/fincrime/prediction_format'
preds_dest_path = 'D:/fincrime-federated/prediction/fincrime/prediction'


## train on data
datapathjsonString = 'data/fincrime/centralized/train/data.json'
swift_data_path = funcs.json_to_dict(datapathjsonString)['swift_data_path']
bank_data_path = funcs.json_to_dict(datapathjsonString)['bank_data_path']

# trainset = funcs.fit(swift_data_path = swift_data_path,
#                     bank_data_path = bank_data_path,
#                     model_dir = model_dir,
#                     preds_format_path = preds_format_path,
#                     preds_dest_path = preds_dest_path,
#                     m = 'xgboost')
def json_to_dict(datapathjsonString):
    datapathJson = open(datapathjsonString)
    datapathDict = json.load(datapathJson)
    return datapathDict


def load_data(swift_data_path, bank_data_path):
    swift_data = pd.read_csv(swift_data_path, index_col="MessageId")
    swift_data["Timestamp"] = swift_data["Timestamp"].astype("datetime64[ns]")
    bank_data = pd.read_csv(bank_data_path)
    return swift_data, bank_data

train_data, bank_data = load_data(
    swift_data_path=swift_data_path, bank_data_path=bank_data_path
)

# Merging with bank details
train_data = pd.merge(
    train_data,
    bank_data,
    left_on="OrderingAccount",
    right_on="Account",
    how="left",
)
train_data = pd.merge(
    train_data,
    bank_data,
    left_on="BeneficiaryAccount",
    right_on="Account",
    how="left",
    suffixes=["_order", "_ben"],
)
trainset = train_data

D:\fincrime-federated


In [3]:
# predict on test data
import submission_src.fincrime.solution_centralized as funcs
importlib.reload(funcs) 

datapathjsonString = 'data/fincrime/centralized/test/data.json'
swift_data_path = funcs.json_to_dict(datapathjsonString)['swift_data_path']
bank_data_path = funcs.json_to_dict(datapathjsonString)['bank_data_path']

# testset = funcs.predict(
#                     swift_data_path =swift_data_path,
#                     bank_data_path = bank_data_path,
#                     model_dir = model_dir,
#                     preds_format_path = preds_format_path,
#                     preds_dest_path = preds_dest_path,
#                     m = 'xgboost'
#                     )   

test_data, bank_data = load_data(
    swift_data_path=swift_data_path, bank_data_path=bank_data_path
)
print(test_data['Sender'].nunique())
# Merging with bank details
test_data = pd.merge(
    test_data,
    bank_data,
    left_on="OrderingAccount",
    right_on="Account",
    how="left",
)
test_data = pd.merge(
    test_data,
    bank_data,
    left_on="BeneficiaryAccount",
    right_on="Account",
    how="left",
    suffixes=["_order", "_ben"],
)

testset = test_data

print("Total time taken ", (time.time() - start_time)," seconds")

16
Total time taken  95.61597180366516  seconds


In [4]:
trainset['Sender'].nunique()

16

In [5]:
# testset.head(1)
testset['Sender'].nunique()

16

In [6]:
trainset.shape

(4691725, 31)

In [7]:
trainsett = trainset.dropna(axis=0,how='any',inplace=True)

In [8]:
testset.shape

(705108, 31)

In [9]:
testset.dropna(axis=0,how='any',inplace=True)
testset.shape

(98118, 31)

In [10]:
trainset['Label'].value_counts()

0    4686615
1       4686
Name: Label, dtype: int64

### Sliced the dataset below (as facing Memory error)

In [11]:
trainset1=trainset.iloc[0:25000,:]

In [12]:
trainset1.shape

(25000, 31)

In [13]:
testset1 = testset.iloc[10000:25000,:]

In [14]:
testset.shape

(98118, 31)

In [15]:
trainset1['Label'].value_counts()

0    24961
1       39
Name: Label, dtype: int64

In [16]:
# swift_bank_df_22=testset.sample(frac=1)

In [17]:
# train_labels = trainset1.pop('Label')
# test_labels = testset1.pop('Label')

In [18]:
def stats(train_dataset):
    train_stats = train_dataset.describe()
    train_stats = train_stats.transpose()
    return train_stats

def norm(x):
    train_stats=stats(x)
    return (x - train_stats['mean']) / train_stats['std']

# Data Prep

In [19]:
def freq_by_features(
    df,
    level_1,
    level_2,
    model_dir,
    save_as_object=False,
    map_from_train_set=False,
) -> None:

    if map_from_train_set == True:

        level_1_level_2_frequency = pickle.load(
            open(
                model_dir
                + "/"
                + str(level_1 + "_" + level_2 + "_frequency")
                + ".sav",
                "rb",
            )
        )
        _level_1 = df.loc[:, level_1].unique()
        _level_2 = df.loc[:, level_2].unique()
        df[str(level_1 + "_" + level_2)] = df.loc[:, level_1] + df.loc[
            :, level_2
        ].astype(str)
        df.loc[:, str(level_1 + "_" + level_2 + "_frequency")] = df.loc[
            :, str(level_1 + "_" + level_2)
        ].map(level_1_level_2_frequency)
    else:
        _level_1 = df.loc[:, level_1].unique()
        _level_2 = df.loc[:, level_2].unique()
        df[str(level_1 + "_" + level_2)] = df.loc[:, level_1] + df.loc[
            :, level_2
        ].astype(str)
        level_1_level_2_frequency = {}
        for s in _level_1:
            level_1_rows = df[df[level_1] == s]
            for h in _level_2:
                level_1_level_2_frequency[s + str(h)] = len(
                    level_1_rows[level_1_rows.loc[:, level_2] == h]
                )

        df.loc[:, str(level_1 + "_" + level_2 + "_frequency")] = df.loc[
            :, str(level_1 + "_" + level_2)
        ].map(level_1_level_2_frequency)

        if save_as_object == True:
            pickle.dump(
                level_1_level_2_frequency,
                open(
                    model_dir
                    + "/"
                    + str(level_1 + "_" + level_2 + "_frequency")
                    + ".sav",
                    "wb",
                ),
            )

    ...

In [20]:
def data_prep1(train_data):
    # Feature engineering
#     train_data = train_data.dropna(axis=0)
    
#     warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
    import warnings

    import pandas as pd
    from pandas.core.common import SettingWithCopyWarning

    warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
    
    train_data = create_features(
        df=train_data, model_dir=model_dir, map_from_train_set=False
    )

    # Keep below columns for training and testing
    cols_to_keep = [
        "SettlementAmount",
        "InstructedAmount",
        "Label",
        "hour",
        "Flags_ben",
        "MissingBenAccount",
        "MissingOrdAccount",
        "Sender_hour_frequency",
        # 'sender_currency_amount_average',
        "Sender_Receiver_frequency",
        "Sender_InstructedCurrency_frequency",
        "seq",
        # 'receiver_transactions',
        "Receiver_SettlementCurrency_frequency",
        "Receiver_hour_frequency",
        "DifferentOrderNum",
        "DifferentBenNum",
        "DifferentOrderName",
        "DifferentBenName",
        "DifferentOrderStreet",
        "DifferentBenStreet",
        "DifferentOrderZip",
        "DifferentBenZip",
    ]

    train_data_2 = train_data[cols_to_keep]
    return train_data_2

In [21]:
def data_prep2(train_data):
    # Feature engineering
#     train_data = train_data.dropna(axis=0)
    train_data = create_features(
        df=train_data, model_dir=model_dir, map_from_train_set=True
    )

    # Keep below columns for training and testing
    cols_to_keep = [
        "SettlementAmount",
        "InstructedAmount",
        "Label",
        "hour",
        "Flags_ben",
        "MissingBenAccount",
        "MissingOrdAccount",
        "Sender_hour_frequency",
        # 'sender_currency_amount_average',
        "Sender_Receiver_frequency",
        "Sender_InstructedCurrency_frequency",
        "seq",
        # 'receiver_transactions',
        "Receiver_SettlementCurrency_frequency",
        "Receiver_hour_frequency",
        "DifferentOrderNum",
        "DifferentBenNum",
        "DifferentOrderName",
        "DifferentBenName",
        "DifferentOrderStreet",
        "DifferentBenStreet",
        "DifferentOrderZip",
        "DifferentBenZip",
    ]

    train_data_2 = train_data[cols_to_keep]
    return train_data_2

In [22]:
def create_features(df, model_dir, map_from_train_set=False):

    ## Feature Engineering

    # Hour column
    df["hour"] = df["Timestamp"].dt.hour

    # Hour frequency for each sender
    freq_by_features(
        df=df,
        level_1="Sender",
        level_2="hour",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )
    # Hour frequency for each receiver
    freq_by_features(
        df=df,
        level_1="Receiver",
        level_2="hour",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )
    # Sender-Currency Frequency
    freq_by_features(
        df=df,
        level_1="Sender",
        level_2="InstructedCurrency",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )
    # Receiver-SettledCurrency Frequency
    freq_by_features(
        df=df,
        level_1="Receiver",
        level_2="SettlementCurrency",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )
    # Sender-Receiver Frequency
    freq_by_features(
        df=df,
        level_1="Sender",
        level_2="Receiver",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )

    # # Average Amount per Sender-Currency - not working
    # Sender_ICurrency_mean = mean_by_features(df = df, level_1="Sender" , level_2="InstructedCurrency")
    # df.loc[:,"Sender_ICurrency_mean"] = \
    #     df.loc[:,"Sender_InstructedCurrency"].map(Sender_ICurrency_mean)

    # Numbering the transactions within a account order - ben - date combination
    df = df.sort_values(
        by=[
            "SettlementDate",
            "Sender",
            "Receiver",
            "Account_order",
            "Account_ben",
            "Timestamp",
        ],
        ascending=True,
    )
    df["seq"] = (
        df.groupby(
            [
                "SettlementDate",
                "Sender",
                "Receiver",
                "Account_order",
                "Account_ben",
            ]
        ).cumcount()
        + 1
    )
    df["seq"] = df["seq"].replace(np.NAN, 1)

    # Flag columns for transactions with missing bank details
    df[["MissingBenAccount"]] = 0
    df.loc[df["Flags_ben"].isnull(), "MissingBenAccount"] = 1
    df[["MissingOrdAccount"]] = 0
    df.loc[df["Flags_order"].isnull(), "MissingOrdAccount"] = 1

    # Different sender account number from bank details
    df["DifferentOrderNum"] = np.where(
        df["Account_order"] == df["OrderingAccount"], 0, 1
    )
    # Different receiver account number from bank details
    df["DifferentBenNum"] = np.where(
        df["Account_ben"] == df["BeneficiaryAccount"], 0, 1
    )

    # Different sender account name from bank details
    df["DifferentOrderName"] = np.where(
        df["Name_order"] == df["OrderingName"], 0, 1
    )
    # Different receiver account name from bank details
    df["DifferentBenName"] = np.where(
        df["Name_ben"] == df["BeneficiaryName"], 0, 1
    )

    # Different sender account ordering street from bank details
    df["DifferentOrderStreet"] = np.where(
        df["Street_order"] == df["OrderingStreet"], 0, 1
    )
    # Different receiver account ordering street from bank details
    df["DifferentBenStreet"] = np.where(
        df["Street_ben"] == df["BeneficiaryStreet"], 0, 1
    )

    # Different sender account country code/zip from bank details
    df["DifferentOrderZip"] = np.where(
        df["CountryCityZip_order"] == df["OrderingCountryCityZip"], 0, 1
    )
    # Different receiver account country code/zip from bank details
    df["DifferentBenZip"] = np.where(
        df["CountryCityZip_ben"] == df["BeneficiaryCountryCityZip"], 0, 1
    )

    # Some missing value treatment
    df.loc[df["Flags_ben"].isna(), "Flags_ben"] = 99

    return df

# Model

In [23]:
# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(28,28)),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(256, activation='relu'),
#     keras.layers.Dense(10, activation='softmax')
# ])
# model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

In [24]:
# import NN layers and other componenets.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
tf.random.set_seed(13) # to make sure the experiment is reproducible.
tf.debugging.set_log_device_placement(False)

def build_model1_two_hidden_layers(normed_train_data):
    # Keras model object created from Sequential class. This will be the container that contains all layers.
    model = Sequential()

    # The model so far is empty. It can be constructed by adding layers and compilation.
    # This Keras model with multiple hidden layers.
    
    # Input Layer with 10 Neurons
    model.add(Dense(32, input_shape = (normed_train_data.shape[1],)))    # Input layer => input_shape must be explicitly designated
#     model.add(Activation('relu')) # relu or sigmoid.
    
#     model.add(Dense(128,Activation('relu')))                         # Hidden layer 1 => only output dimension should be designated (output dimension = # of Neurons = 50)
    
    
    
    
    model.add(Dense(1))                          # Output layer => output dimension = 1 since it is a regression problem
    
    # Activation: sigmoid, softmax, tanh, relu, LeakyReLU. 
    #Optimizer: SGD, Adam, RMSProp, etc. # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
    learning_rate = 0.0001
#     optimizer = optimizers.Nadam(learning_rate)
    optimizer = 'adam'
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy',tf.keras.metrics.Precision(),
                                                                       tf.keras.metrics.Recall()]) 
    # for regression problems, mean squared error (MSE) is often employed
    
    return model

# Fed Learning

In [25]:
from collections import OrderedDict
from typing import List, Tuple

import flwr as fl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split

In [26]:
DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.13.0+cpu and Flower 1.1.0


In [27]:
len(testset)

98118

#### Split into 10 clients

In [28]:
NUM_CLIENTS=2
BATCH_SIZE = 128

In [29]:
len(trainset)

4691301

In [30]:
# from sklearn.model_selection import train_test_split

# def load_datasets():
#     # Download and transform CIFAR-10 (train and test)

#     trainloadset,valloadset=train_test_split(trainset,test_size=0.3)
#     testset2=testset
#     print("length of Trainset=",len(trainloadset))
#     print("length of Valset=",len(valloadset))
#     print("length of Testset=",len(testset2))
#     le1=len(trainloadset)//10
#     le2=len(valloadset)//10
    
#     trainloaders=[]
#     valloaders=[]
#     for i in range(1,NUM_CLIENTS+1):
#         temp_df1=trainloadset.head(le1)
#         trainloadset=trainloadset.tail(len(trainloadset)-le1)
#         temp_df2=valloadset.head(le2)
#         valloadset=valloadset.tail(len(valloadset)-le2)
#         trainloaders.append(temp_df1)
#         valloaders.append(temp_df2)
    
        
    
#     return trainloaders, valloaders, testset2

# trainloaders, valloaders, testloader = load_datasets()

In [31]:
trainloaders=[]
valloaders=[]

from sklearn.model_selection import train_test_split
trainloadset,valloadset=train_test_split(trainset,test_size=0.3)

trainloaders.append(trainloadset[trainloadset['Sender']=='DPSUFRPP'])
trainloaders.append(trainloadset[trainloadset['Sender']!='DPSUFRPP'])

valloaders.append(valloadset[valloadset['Sender']=='DPSUFRPP'])
valloaders.append(valloadset[valloadset['Sender']!='DPSUFRPP'])

testloader=testset

In [32]:
trainset.shape

(4691301, 31)

In [33]:
(trainloaders[0].shape)

(1375359, 31)

In [34]:
testloader.shape

(98118, 31)

In [36]:
from sklearn.utils import class_weight

In [37]:
def train(model, trainloader,valloader,train_labels,valid_labels, epochs: int, verbose=False):
    """Train the network on the training set."""
    batch_size=128
    EPOCHS=2
    normed_train_data=trainloader
    normed_valid_dataset=valloader
    
    class_weights = class_weight.compute_class_weight('balanced',np.unique(train_labels),train_labels)

    history = model.fit(
        normed_train_data, 
        train_labels,
        batch_size = batch_size,
        epochs=EPOCHS, 
        verbose=1,
        shuffle=True,
        steps_per_epoch = int(normed_train_data.shape[0] / batch_size) ,
        validation_data = (normed_valid_dataset, valid_labels),   
        class_weight=class_weights,
    )
    


def test(model, testloader,test_labels):
    normed_test_data=testloader
    """Evaluate the network on the entire test set."""
    print('Test Split: ')
    results =  model.evaluate(normed_test_data, test_labels, verbose=2)
    print(model.metrics_names)
    print(results)
    #print("Accuracy   : {:5.2f} ".format(accuracy))


from sklearn.metrics import (accuracy_score,classification_report,confusion_matrix)
from sklearn import metrics

def score_data(model, X_train_data, Y_train_data) -> None:

#     pred = model.predict(X_train_data[:1])
    scaler.fit(X_train_data)
    Xnew = scaler.transform(X_train_data)
    
    y_pred = model.predict(Xnew)
    ynew = np.round(y_pred).astype(int)
    print("AUPRC:", metrics.average_precision_score(y_true=Y_train_data, y_score=ynew))
    
#     prediction = model.predict(x_test[:1])
    print("prediction shape:", pred.shape)


In [38]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(quantile_range=(25, 75))

### Centralized result using Keras

In [39]:
tl=data_prep1(trainloaders[0])
vl=data_prep1(valloaders[0])
testl=data_prep1(testloader)
train_labels=tl.pop('Label')
val_labels=vl.pop('Label')
test_labels=testl.pop('Label')
trainloader=(tl)
valloader=(vl)
testloader=(testl)
model=build_model1_two_hidden_layers(trainloader)
model.save("my_model")
# print(trainloader)

train(model, trainloader,valloader,train_labels,val_labels, 1)

test(model, testloader,test_labels)

score_data(model,testloader,test_labels)

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets


TypeError: compute_class_weight() takes 1 positional argument but 3 were given

In [ ]:
test_labels.value_counts()

In [ ]:
tl.head()

#### Alternate split

In [ ]:
tl=data_prep1(trainloaders[1])
vl=data_prep1(valloaders[1])
# testl=data_prep1(testloader)
train_labels=tl.pop('Label')
val_labels=vl.pop('Label')
# test_labels=testl.pop('Label')
trainloader=(tl)
valloader=(vl)
# testloader=(testl)
train(model, trainloader,valloader,train_labels,val_labels, 1)

test(model, testloader,test_labels)

score_data(model,testloader,test_labels)

### Centralized results using sklearn

In [ ]:
train_set=data_prep1(trainset)

# y_train = train_set.pop('Label')
y_train = train_set["Label"].values
X_train = train_set.drop(["Label"], axis=1).values


In [ ]:
def train_model(X_train_data, Y_train_data, m) -> None:
    if m == "rf":
        model = RandomForestClassifier(max_depth=7, random_state=0, n_estimators=10)
    elif m == "xgboost":
        model = XGBClassifier(n_estimators=100)

    kfold = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
#     cv_results = cross_val_score(model, X_train_data, Y_train_data, cv=kfold, scoring="f1")
    model.fit(X_train_data, Y_train_data)
#     print("Minimum:", cv_results.min())
#     print("Maximum:", cv_results.max())
#     print("StanDev:", cv_results.std())
    return model

def scoredata(X_train_data, Y_train_data, model, m) -> None:
    pred = model.predict(X_train_data)
#     print(m + " Classification Report=\n\n",classification_report(Y_train_data, pred))
#     print(m + " Confusion Matrix=\n\n", confusion_matrix(Y_train_data, pred))
    pred_proba = model.predict_proba(X_train_data)[:, 1]
    print("AUPRC:",metrics.average_precision_score(y_true=Y_train_data, y_score=pred_proba),)
    return pred, pred_proba

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBClassifier
import sklearn.utils
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import (
    KFold,
    ShuffleSplit,
    StratifiedKFold,
    StratifiedShuffleSplit,
    cross_val_score,
    train_test_split,
)

In [ ]:
# from sklearn.preprocessing import QuantileTransformer
# scaler = QuantileTransformer(output_distribution="uniform")

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(quantile_range=(25, 75))
scaler.fit(X_train)
tl = scaler.transform(X_train)
test_loader = scaler.transform(testl)

In [ ]:
#Fit the model on data        train_model(X_train_data=tl, Y_train_data=y_train, m='rf')
model = RandomForestClassifier(max_depth=7, random_state=0, n_estimators=10)
# kfold = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
# cv_results = cross_val_score(model, X_train_data=tl, Y_train_data=y_train, cv=kfold, scoring="f1")
model.fit(tl, y_train)

In [ ]:
# test_loader = scaler.transform(testl)
pred, pred_proba = scoredata(X_train_data=test_loader,Y_train_data=test_labels,model=model,m='rf')

In [ ]:
pred

In [ ]:
from sklearn.linear_model import LogisticRegression

# MNist Client

In [ ]:
from typing import Tuple, Union, List
XY = Tuple[np.ndarray, np.ndarray]
Dataset = Tuple[XY, XY]
LogRegParams = Union[XY, Tuple[np.ndarray]]
XYList = List[XY]

def get_model_parameters(model: LogisticRegression) -> LogRegParams:
    """Returns the paramters of a sklearn LogisticRegression model."""
    if model.fit_intercept:
        params = [
            model.coef_,
            model.intercept_,
        ]
    else:
        params = [
            model.coef_,
        ]
    return params

def set_model_params(
    model: LogisticRegression, params: LogRegParams
) -> LogisticRegression:
    """Sets the parameters of a sklean LogisticRegression model."""
    model.coef_ = params[0]
    if model.fit_intercept:
        model.intercept_ = params[1]
    return model

In [ ]:
# Define Flower client
class MnistClient(fl.client.NumPyClient):
    def get_parameters(self, config):  # type: ignore
        return utils.get_model_parameters(model)

    def fit(self, parameters, config):  # type: ignore
        utils.set_model_params(model, parameters)
        # Ignore convergence failure due to low local epochs
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model.fit(X_train, y_train)
        print(f"Training finished for round {config['server_round']}")
        return utils.get_model_parameters(model), len(X_train), {}

    def evaluate(self, parameters, config):  # type: ignore
        utils.set_model_params(model, parameters)
        loss = log_loss(y_test, model.predict_proba(X_test))
        accuracy = model.score(X_test, y_test)
        return loss, len(X_test), {"accuracy": accuracy}

In [ ]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)# class SaveModelStrategy(fl.server.strategy.FedAvg):


## NumpyClient

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, trainloader, valloader):
        self.model = model
        self.trainloader = trainloader
        self.valloader = valloader
        
    def get_parameters(self,config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        tl=data_prep1(self.trainloader)
        vl=data_prep1(self.valloader)
        train_labels=tl.pop('Label')
        val_labels=vl.pop('Label')
        test_labels=testl.pop('Label')
        trainloader=(tl)
        valloader=(vl)
        train(self.model,trainloader,valloader,train_labels,val_labels, 1)
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        results = model.evaluate(testloader, test_labels, verbose=0)
#         loss, accuracy = model.evaluate(testloader, test_labels, verbose=0)
#         results =  model.evaluate(normed_test_data, test_labels, verbose=2)
        pred = model.predict(test_loader)
#         print(pred)
        auprc = metrics.average_precision_score(y_true=test_labels, y_score=pred)
        print("AUPRC:", auprc)
        print("Eval accuracy : ", results[1])
        loss = results[0]
        return float(loss), len(x_test), {"accuracy": results[1]}

In [ ]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    loaded_model = keras.models.load_model("my_model")

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(loaded_model, trainloader, valloader)

In [ ]:
NUM_CLIENTS

In [ ]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
        min_fit_clients=2,  # Never sample less than 10 clients for training
        min_evaluate_clients=1,  # Never sample less than 5 clients for evaluation
        min_available_clients=2,  # Wait until all 10 clients are available
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=2,
    config=fl.server.ServerConfig(num_rounds=5),
    #strategy=strategy,
)

## Weighted average

In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [ ]:
strategy = fl.server.strategy.FedAvg(
        fraction_fit=0.8,
        fraction_evaluate=0.5,
        min_fit_clients=2,
        min_evaluate_clients=1,
        min_available_clients=2,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
)

## FedAdagrad

In [ ]:
# Create FedAdam strategy
strategy=fl.server.strategy.FedAdagrad(
    fraction_fit=1,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

In [ ]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 2,
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=5,
    min_evaluate_clients=10,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

## FedCustom

In [ ]:
from typing import Callable, Union
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy


class FedCustom(fl.server.strategy.Strategy):
    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # TODO WIP - add implementation

        return []

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        # TODO WIP - add implementation

        return None, {}

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""

        # TODO WIP - add implementation

        return []

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        # TODO WIP - add implementation

        return None, {}

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate model parameters using an evaluation function."""

        # TODO WIP - add implementation

        return None

In [ ]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=FedCustom(),  # <-- pass the new strategy here
)

## Client

In [ ]:
from flwr.common import Code, EvaluateIns, EvaluateRes, FitIns, FitRes, GetParametersIns, GetParametersRes, Status
from flwr.common import ndarrays_to_parameters, parameters_to_ndarrays


class FlowerClient(fl.client.Client):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, ins: GetParametersIns) -> GetParametersRes:
        print(f"[Client {self.cid}] get_parameters")

        # Get parameters as a list of NumPy ndarray's
        ndarrays: List[np.ndarray] = get_parameters(self.net)

        # Serialize ndarray's into a Parameters object
        parameters = ndarrays_to_parameters(ndarrays)

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return GetParametersRes(
            status=status,
            parameters=parameters,
        )

    def fit(self, ins: FitIns) -> FitRes:
        print(f"[Client {self.cid}] fit, config: {ins.config}")

        # Deserialize parameters to NumPy ndarray's
        parameters_original = ins.parameters
        ndarrays_original = parameters_to_ndarrays(parameters_original)

        # Update local model, train, get updated parameters
        set_parameters(self.net, ndarrays_original)
        train(self.net, self.trainloader, epochs=1)
        ndarrays_updated = get_parameters(self.net)

        # Serialize ndarray's into a Parameters object
        parameters_updated = ndarrays_to_parameters(ndarrays_updated)

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return FitRes(
            status=status,
            parameters=parameters_updated,
            num_examples=len(self.trainloader),
            metrics={},
        )

    def evaluate(self, ins: EvaluateIns) -> EvaluateRes:
        print(f"[Client {self.cid}] evaluate, config: {config}")

        # Deserialize parameters to NumPy ndarray's
        parameters_original = ins.parameters
        ndarrays_original = parameters_to_ndarrays(parameters_original)

        set_parameters(self.net, ndarrays_original)
        loss, accuracy = test(self.net, self.valloader)
        # return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return EvaluateRes(
            status=status,
            loss=float(loss),
            num_examples=len(self.valloader),
            metrics={"accuracy": float(accuracy)},
        )


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [ ]:
class FlowerNumPyClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def numpyclient_fn(cid) -> FlowerNumPyClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerNumPyClient(cid, net, trainloader, valloader)

In [ ]:
fl.simulation.start_simulation(
    client_fn=numpyclient_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
)